# 用收益率曲线斜率预测经济衰退概率

如何用收益率曲线斜率来预测美国经济衰退的概率？

1. 获取债券收益率数据，包括3个月短期国库券收益率和10年期国债收益率。
2. 获取NBER经济衰退指标，这个指标包含两个值，1表示经济衰退，0表示经济扩张。
3. 估计Probit模型，预测变量是收益率曲线的斜率（即10年期国债和3个月国库券的利差），目标变量是未来1年经济衰退的概率。

估计模型：$P(NBER_{t+i,t+j}=1) = \phi(\beta_0+\beta_1Spread_t)$

- $NBER_{t+i,t+j}=1$ 表示NBER衰退指标在 $t+i$ 和 $t+j$ 时间段内显示经济衰退。
- $Spread_t$ 表示10年期国债和3个月国库券利差。

参考资料：

* [Predicting Recession Probabilities Using the Slope of the Yield Curve](https://www.federalreserve.gov/econres/notes/feds-notes/predicting-recession-probabilities-using-the-slope-of-the-yield-curve-20180301.html)

什么是NBER衰退指标？

- NBER(美国国家经济研究局)
- NBER经济衰退指标是由美国国家经济研究局创建的用于确定经济衰退和扩张周期的指标。
- 衰退指标是月数据，仅包含两个值，1和0，1表示衰退时期，0表示扩张时期。
- NBER不仅依赖GDP来判断经济衰退，而是考虑一系列指标，包括实际个人收入，非农就业增长和工业产生等。
- NBER强调衰退意味着经济活动全面降温，涉及很多领域且持续至少数月的时间。

In [11]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from fredapi import Fred
from dotenv import load_dotenv

plt.style.use("ggplot")

从FRED下载利率数据。

In [2]:
# Prepare FRED API key 
load_dotenv()
fred_api_key = os.getenv('FRED_APIKEY')
fred = Fred(fred_api_key)

# List of rates to download
rates = [
    'DTB3',   # 3-month Treasury Bill
    'DTB6',   # 6-month Treasury Bill
    'DGS5',   # 5-year Treasury Bond
    'DGS10',  # 10-year Treasury Bond
    'DGS30'   # 30-year Treasury Bond
]

# Start and end dates
start = '1950-01-01'  
end = '2023-09-30'

# Initialize dataframe
df = pd.DataFrame()

# Download each rate and add to dataframe
for rate in rates:
    data = fred.get_series(rate, observation_start=start, observation_end=end)
    data.name = rate
    df = pd.concat([df, data], axis=1)

# Save to CSV file
df.to_csv('./data/US_interest_rates.csv')

下载NBER衰退指标数据。

In [3]:
nber_recession_indicator = fred.get_series('USREC', observation_start=start, observation_end=end)
nber_recession_indicator.to_csv('./data/US_recession_indicator.csv')

读取数据，清洗和整合数据。

In [4]:
rates = pd.read_csv('./data/US_interest_rates.csv', index_col=0, parse_dates=True)
rec = pd.read_csv('./data/US_recession_indicator.csv', index_col=0, parse_dates=True)

In [5]:
rates.tail()

,DTB3,DTB6,DGS5,DGS10,DGS30
2023-09-25,5.35,5.31,4.62,4.55,4.67
2023-09-26,5.34,5.32,4.62,4.56,4.70
2023-09-27,5.34,5.33,4.67,4.61,4.73
2023-09-28,5.32,5.32,4.62,4.59,4.71
2023-09-29,5.32,5.32,4.60,4.59,4.73


In [6]:
rec.tail()

,0
2023-05-01,0.0
2023-06-01,0.0
2023-07-01,0.0
2023-08-01,0.0
2023-09-01,0.0


In [7]:
data = (
    rates.dropna()
    .resample("MS").mean()
    .assign(Spread=lambda x: x.DGS10 - x.DTB3)
    .assign(Recession=lambda x: rec)
)
data

,DTB3,DTB6,DGS5,DGS10,DGS30,Spread,Recession
1977-02-01,4.672222,4.893333,6.854444,7.411111,7.754444,2.738889,0.0
1977-03-01,4.602609,4.878261,6.928261,7.460000,7.796087,2.857391,0.0
1977-04-01,4.535500,4.798000,6.792000,7.370500,7.734500,2.835000,0.0
1977-05-01,4.956190,5.200000,6.939048,7.456667,7.798095,2.500476,0.0
1977-06-01,5.016364,5.211818,6.756818,7.275455,7.635909,2.259091,0.0
...,...,...,...,...,...,...,...
2023-05-01,5.135455,5.025000,3.591364,3.573636,3.855455,-1.561818,0.0
2023-06-01,5.158095,5.191905,3.949524,3.748095,3.867619,-1.410000,0.0
2023-07-01,5.254000,5.268000,4.141500,3.899500,3.959000,-1.354500,0.0
2023-08-01,5.296522,5.284348,4.306522,4.167826,4.280000,-1.128696,0.0


数据可视化，观察收益率曲线斜率和NBER衰退指标的关系。

In [8]:
# Get recession periods
recession = data.Recession
recession_diff = recession.diff()
recession_start_dates = recession_diff.index[recession_diff == 1]
recession_end_dates = recession_diff.index[recession_diff == -1]    

In [9]:
fig = go.Figure()

fig.add_scatter(x=data.index, y=data.Spread, name="Yield curve spread", mode="lines")

for start_date, end_date in zip(recession_start_dates, recession_end_dates):
    fig.add_vrect(
        x0=start_date, x1=end_date,
        fillcolor="red", opacity=0.25,
        layer="below", line_width=0,
    )

fig.update_layout(width=1000, height=618, title="Yield curve and recessions")
fig.show()

估计Probit模型，预测未来1年经济衰退的概率。

首先要创建一个新指标作为目标变量，这个指标基于NBER衰退指标，表示未来1年内是否会出现经济衰退。

In [17]:
def determine_future_recession(recession: pd.Series,
                               i: int = 1,
                               j: int = 12) -> pd.Series:
    """Determine if a recession will happen in the future.
    
    Args:
        recession (pd.Series): Recession indicator.
        i (int, optional): Number of months ahead to look. Defaults to 1.
        j (int, optional): Number of months to look ahead. Defaults to 12.        
    """
    future_recession = np.zeros(len(recession))
    for t in range(len(recession)):
        if recession.iloc[t+i:t+j+1].sum() > 0:
            future_recession[t] = 1
    return future_recession

In [23]:
data["future_recession"] = determine_future_recession(data.Recession)
data.head()

,DTB3,DTB6,DGS5,DGS10,DGS30,Spread,Recession,future_recession
1977-02-01,4.672222,4.893333,6.854444,7.411111,7.754444,2.738889,0.0,0.0
1977-03-01,4.602609,4.878261,6.928261,7.460000,7.796087,2.857391,0.0,0.0
1977-04-01,4.535500,4.798000,6.792000,7.370500,7.734500,2.835000,0.0,0.0
1977-05-01,4.956190,5.200000,6.939048,7.456667,7.798095,2.500476,0.0,0.0
1977-06-01,5.016364,5.211818,6.756818,7.275455,7.635909,2.259091,0.0,0.0


估计Probit模型

In [24]:
from statsmodels.discrete.discrete_model import Probit
from statsmodels.tools.tools import add_constant

In [27]:
# Prepare data
X = add_constant(data.Spread)
y = data.future_recession

# Fit model
model = Probit(y, X)
probit_model = model.fit()

# Print summary
print(probit_model.summary())

# Get marginal effects
marginal_effects = probit_model.get_margeff()
print(marginal_effects.summary())

Optimization terminated successfully.
         Current function value: 0.455446
         Iterations 6
                          Probit Regression Results                           
Dep. Variable:       future_recession   No. Observations:                  560
Model:                         Probit   Df Residuals:                      558
Method:                           MLE   Df Model:                            1
Date:                Sat, 07 Oct 2023   Pseudo R-squ.:                  0.1386
Time:                        11:57:40   Log-Likelihood:                -255.05
converged:                       True   LL-Null:                       -296.08
Covariance Type:            nonrobust   LLR p-value:                 1.323e-19
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1494      0.092     -1.622      0.105      -0.330       0.031
Spread        -0.4401      0.

结果：

* 利率差异系数小于0，表示利差上升会减少经济衰退的概率，利差下降会增加衰退概率，这与我们的预期相符。
* 利差系数统计显著，说明利差是一个有效的预测变量。
* 利率差异减少1%，未来1年经济发生衰退的概率上升11.3%。

计算预测概率，将预测概率与实际发生衰退的时间进行比较。

In [30]:
recession_probs = probit_model.predict(X)

fig = go.Figure()

fig.add_scatter(x=recession_probs.index, y=recession_probs, name="Recession probability", mode="lines")

for start_date, end_date in zip(recession_start_dates, recession_end_dates):
    fig.add_vrect(
        x0=start_date, x1=end_date,
        fillcolor="red", opacity=0.25,
        layer="below", line_width=0,
    )

fig.update_layout(width=1000, height=618, title="Probability of recession from probit model")
fig.show()

将模型结果与美联储研究文献的结果进行比较。

如何计算置信区间？将95%置信区间添加到图表。